Installers<br>
pandas: !pip install pandas<br>
K-means: !pip install -U scikit-learn<br>
Matplotlib: !python -m pip install -U matplotlib<br>
nltk: !pip install nltk<br>
PySimpleGUI: !pip install pysimplegui

In [2]:
from asyncore import write
import pandas as pd
from doctest import DocTestFailure
import math
import string
from sklearn.cluster import KMeans
from sklearn.datasets._samples_generator import make_blobs

#----------- importando bibliotecas para tratar as palavras

#from worldcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from nltk.tokenize import word_tokenize
import nltk

#----------- Biblioteca para interface
import PySimpleGUI as sg

#----------- Ignorar Warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
#----------- funções

def remove_punctuations(line):
    for character in string.punctuation:
        line = line.replace(character, "")
    return line

def writeFile(dict, archiveName):
    arquivo = open(archiveName+".txt",'w')
    for word, tf in dict.items():
        sentence = f"{word}:{tf}\n"
        arquivo.write(sentence)
    arquivo.close()

def writeCSV(df, archiveName):
    f = open("..\\DataBase\\"+archiveName+".csv", 'w', newline='', encoding="utf8")
    df.to_csv("..\\DataBase\\"+archiveName+".csv", encoding="utf8", sep=';')


def countWords(wordsDoc, line): 
    wordsCopy = wordsDoc.copy() 
    words = line.split()
    for word in words:
        word = word.lower()
        wordsCopy[word] += 1
    return wordsCopy

def get_posts():
    file = open(filepath, 'r', encoding="utf8")
    lines = file.readlines()
    return lines  

In [4]:
#------------- funções de ponderação/vetorização

def computeTF(wordDict, doc):
    tfDict = {}
    corpusCount = len(doc)
    for word, count in wordDict.items():
        tfDict[word] = round(count/float(corpusCount), 3)
    return(tfDict)

def computeIDF(docList):
    idfDict = {}
    N = len(docList)
    
    #idfDict = dict.fromkeys(docList[1].keys, 0)
    for word, val in docList.items():
        idfDict[word] = round(math.log10(N / (float(val) + 1)), 3)
        
    return(idfDict)

def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = round(val*idfs[word], 3)
    return(tfidf)

In [5]:
#--------- pegando cada palavra dos posts 

filepath = "..\\DataBase\\Governamentais.txt"
word_count = {}
lines = []


with open(filepath, 'r', encoding="utf8") as fi:
    for line in fi:
        line = remove_punctuations(line)
        lines.append(line)
        words = line.split()

        for word in words:
            word = word.lower()
            if word not in word_count:
                word_count[word] = 0

In [6]:
#-----criando listas auxiliares
listTF = {}
listIDF = {}
listTF_IDF = {}
tweets = {}

In [7]:
#-----ponderação/vetorização
rowCount = 0
for line in lines:
    rowCount += 1
    rowName = str(rowCount)
    totalCount = countWords(word_count, line)
    totalTF = computeTF(totalCount, line)
    listTF[rowName] = totalTF
    totalIDF = computeIDF(totalCount)
    listIDF[rowName] = totalIDF
    totalTF_IDF = computeTFIDF(totalCount, totalIDF)
    listTF_IDF[rowName] = totalTF_IDF
    tweets[rowName] = line

In [32]:
#------------ criando os dataframes

dfTF = pd.DataFrame.from_dict(listTF, orient='index', columns=word_count.keys())
dfIDF = pd.DataFrame.from_dict(listIDF, orient='index', columns=word_count.keys())
dfTF_IDF = pd.DataFrame.from_dict(listTF_IDF, orient='index', columns=word_count.keys())
dfTweets = pd.DataFrame.from_dict(tweets, orient='index', columns=['TEXT'])


In [9]:
#---------clusterização --- TF
def clusterizar(dfTF):
    x = np.array(dfTF)
    kmeans_model = KMeans (n_clusters=3, random_state=0, init='random', n_init=10, max_iter=300, tol=1e-04).fit(x)
    kmeans_model.labels_
    kmeans_model.cluster_centers_

    mydict = {i: np.where(kmeans_model.labels_ == i)[0] for i in range(kmeans_model.n_clusters)}
    dictlist = []
    for key, value in mydict.items():
        temp = [key,value]
        dictlist.append(temp)
        
    return dictlist

### Inserir coluna do grupo

In [10]:
#--------- Função para inserir coluna dos grupos
def insert_column(df_aux,dictlist):
    df_aux['group'] = ''
    for i in dictlist[0][1]:
        df_aux['group'][i-1] = 0

    for i in dictlist[1][1]:
        df_aux['group'][i-1] = 1

    for i in dictlist[2][1]:
        df_aux['group'][i-1] = 2
    
    return df_aux

### Inserir coluna da classificação

In [11]:
#--------- Função para inserir coluna dos grupos
def insert_column_class(dfTF,df_aux,sentimento):
    for i in df_aux.index:
        dfTF['classification'][int(i)-1] = sentimento
    
    return dfTF

In [ ]:
#df_aux = dfTF
#df_aux['group'] = ''
   
#df_aux = insert_column(df_aux)
#display(df_aux)

## Classificar as amostras

### Função para classificação

In [12]:
# A função retorna verdadeiro caso todas as amostras sejam classificadas igualmente
def classification(amostra,lines):    
    sg.theme('DarkAmber')

    first = True
    equals = True

    for i in amostra.index:
        layout = [  [sg.Text(lines[int(i)-1])],
                    [sg.Button('Positivo'), sg.Button('Negativo'), sg.Button('Neutro')] ]

        window = sg.Window('Classificação de Post', layout)

        event, values = window.read()
        if(event == sg.WIN_CLOSED):
            window.close()
            break
        elif(first):
            classification = event
            first = False
            window.close()
        elif(classification != event):
            equals = False
            window.close()
            break
        else:
            window.close()
    #window.close()    
    return equals, classification


### Pegar posts como amostra de cada grupo

In [13]:
def get_group(grupo, df_aux):
    return df_aux[df_aux['group'] == grupo]

### Pega os posts em txt

In [14]:
lines = get_posts()

### Classificar Grupo 0

In [34]:
dictlist = clusterizar(dfTF)
df_aux = insert_column(dfTF, dictlist)
gp0 = get_group(0,df_aux)
gp1 = get_group(1,df_aux)
gp2 = get_group(2,df_aux) 
if(len(gp0) > 5): #Caso a quantidade de posts no grupo seja inferior a 5, a quantidade de SAMPLE será igual a quantidade de posts
    result, classific = classification(gp0.sample(5), lines) #Classifica o Grupo 0
else:
    result, classific = classification(gp0.sample(len(gp0)), lines) #Classifica o Grupo 0
print(result)

False


In [35]:
dfTF['classification'] = ''

Repete o código abaixo enquanto der False

In [36]:
dictlist = clusterizar(gp0)
df_aux = insert_column(gp0, dictlist)
gp0 = get_group(0,df_aux)
gp1 = pd.concat([gp1,get_group(1,df_aux)])
gp2 = pd.concat([gp2,get_group(2,df_aux)])
if(len(gp0) > 5): #Caso a quantidade de posts no grupo seja inferior a 5, a quantidade de SAMPLE será igual a quantidade de posts
    result, classific = classification(gp0.sample(5), lines) #Classifica o Grupo 0
else:
    result, classific = classification(gp0.sample(len(gp0)), lines) #Classifica o Grupo 0
print(result)

True


### Tudo OK

Execute este código apenas quando o grupo 0 estiver OK, lembrando de alterar o nome do DF na função abaixo para o ultimo grupo 0 que você criou

In [37]:
dfTF = insert_column_class(dfTF, gp0, classific)
gp0 = pd.DataFrame()
dfTF

,telejornal,traz,uma,entrevista,ao,vivo,com,o,candidato,governo,...,okapanegão,grpaesp,baracho,pmesp,cmt,brpae,sjrp,httpdlvritjtsdk,group,classification
1,0.009,0.009,0.009,0.009,0.017,0.009,0.009,0.009,0.009,0.009,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,
2,0.000,0.000,0.005,0.000,0.000,0.000,0.005,0.005,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,
3,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,
4,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.007,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,
5,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
842,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.021,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,
843,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,
844,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,
845,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,


## Classificar grupo 1

In [39]:
if(len(gp1) > 0):
    print(True)
else:
    print(False)

True


Primeira vez na classificação do Grupo 1, não clusteriza

In [40]:
if(len(gp1) > 5): #Caso a quantidade de posts no grupo seja inferior a 5, a quantidade de SAMPLE será igual a quantidade de posts
    result, classific = classification(gp1.sample(5), lines) #Classifica o Grupo 0
else:
    result, classific = classification(gp1.sample(len(gp1)), lines) #Classifica o Grupo 0
print(result)
if(result):
    print('Vá direto para o grupo 2. Não precisa fazer do grupo 0 nesta etapa')

True
Vá direto para o grupo 2. Não precisa fazer do grupo 0 nesta etapa


Na Segunda vez, o grupo 0 vai ser sobrescrevido, pois agora será formado um novo grupo 0

In [ ]:
dictlist = clusterizar(gp1)
df_aux = insert_column(gp1, dictlist)
gp0 = get_group(0,df_aux)
gp1 = get_group(1,df_aux)
gp2 = pd.concat([gp2,get_group(2,df_aux)])
if(len(gp1) > 5): #Caso a quantidade de posts no grupo seja inferior a 5, a quantidade de SAMPLE será igual a quantidade de posts
    result, classific = classification(gp1.sample(5), lines) #Classifica o Grupo 0
else:
    result, classific = classification(gp1.sample(len(gp1)), lines) #Classifica o Grupo 0
print(result)

A partir da terceira, os grupos 0 e 2 deverão ser concatenados

In [ ]:
if(len(gp1) >= 3):
    dictlist = clusterizar(gp1)
    df_aux = insert_column(gp1, dictlist)
    gp0 = pd.concat([gp0,get_group(0,df_aux)])
    gp1 = get_group(1,df_aux)
    gp2 = pd.concat([gp2,get_group(2,df_aux)])
    if(len(gp1) > 5): #Caso a quantidade de posts no grupo seja inferior a 5, a quantidade de SAMPLE será igual a quantidade de posts
        result, classific = classification(gp1.sample(5), lines) #Classifica o Grupo 0
    else:
        result, classific = classification(gp1.sample(len(gp1)), lines) #Classifica o Grupo 0
    print(result)
else:
    if(len(gp1) > 5): #Caso a quantidade de posts no grupo seja inferior a 5, a quantidade de SAMPLE será igual a quantidade de posts
        result, classific = classification(gp1.sample(5), lines) #Classifica o Grupo 0
    else:
        result, classific = classification(gp1.sample(len(gp1)), lines) #Classifica o Grupo 0
    print(result)

### Tudo OK

In [41]:
dfTF = insert_column_class(dfTF, gp1, classific)
gp1 = pd.DataFrame()
dfTF

,telejornal,traz,uma,entrevista,ao,vivo,com,o,candidato,governo,...,okapanegão,grpaesp,baracho,pmesp,cmt,brpae,sjrp,httpdlvritjtsdk,group,classification
1,0.009,0.009,0.009,0.009,0.017,0.009,0.009,0.009,0.009,0.009,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,
2,0.000,0.000,0.005,0.000,0.000,0.000,0.005,0.005,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,
3,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,
4,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.007,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,
5,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
842,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.021,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,
843,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,
844,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,
845,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,


### Novo Grupo 0 Tem posts? 

Caso tenha, deve voltar e classificar o novo grupo 0

In [88]:
if(len(gp0) > 0):
    print(True)
else:
    print(False)

False


Primeira vez na classificação do Grupo 0, não clusteriza

In [74]:
if(len(gp0) > 5): #Caso a quantidade de posts no grupo seja inferior a 5, a quantidade de SAMPLE será igual a quantidade de posts
    result, classific = classification(gp0.sample(5), lines) #Classifica o Grupo 0
else:
    result, classific = classification(gp0.sample(len(gp0)), lines) #Classifica o Grupo 0
print(result)

False


Se tiver vindo do grupo 1 execute o código abaixo.

In [75]:
if(len(gp0) >= 3):
    dictlist = clusterizar(gp0)
    df_aux = insert_column(gp0, dictlist)
    gp0 = get_group(0,df_aux)
    gp1 = get_group(1,df_aux)
    gp2 = pd.concat([gp2,get_group(2,df_aux)])## Classificar grupo 1

    if(len(gp0) > 5): #Caso a quantidade de posts no grupo seja inferior a 5, a quantidade de SAMPLE será igual a quantidade de posts
        result, classific = classification(gp0.sample(5), lines) #Classifica o Grupo 0
    else:
        result, classific = classification(gp0.sample(len(gp0)), lines) #Classifica o Grupo 0
    print(result)
else:
    if(len(gp0) > 5): #Caso a quantidade de posts no grupo seja inferior a 5, a quantidade de SAMPLE será igual a quantidade de posts
        result, classific = classification(gp0.sample(5), lines) #Classifica o Grupo 0
    else:
        result, classific = classification(gp0.sample(len(gp0)), lines) #Classifica o Grupo 0
    print(result)

True


Se tiver vindo do grupo 2 execute o código abaixo.

In [67]:
dictlist = clusterizar(gp0)
df_aux = insert_column(gp0, dictlist)
gp0 = get_group(0,df_aux)
gp1 = pd.concat([gp1,get_group(1,df_aux)])
gp2 = pd.concat([gp2,get_group(2,df_aux)])
if(len(gp0) > 5): #Caso a quantidade de posts no grupo seja inferior a 5, a quantidade de SAMPLE será igual a quantidade de posts
    result, classific = classification(gp0.sample(5), lines) #Classifica o Grupo 0
else:
    result, classific = classification(gp0.sample(len(gp0)), lines) #Classifica o Grupo 0

print(result)

True


### Tudo OK GP 0

In [76]:
dfTF = insert_column_class(dfTF, gp0, classific)
gp0 = pd.DataFrame()
dfTF

,telejornal,traz,uma,entrevista,ao,vivo,com,o,candidato,governo,...,okapanegão,grpaesp,baracho,pmesp,cmt,brpae,sjrp,httpdlvritjtsdk,group,classification
1,0.009,0.009,0.009,0.009,0.017,0.009,0.009,0.009,0.009,0.009,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
2,0.000,0.000,0.005,0.000,0.000,0.000,0.005,0.005,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
3,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
4,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.007,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
5,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
842,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.021,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
843,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
844,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
845,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Neutro


### Tem posts no grupo 1?

In [89]:
if(len(gp1) > 0):
    print(True)
else:
    print(False)

False


In [78]:
if(len(gp1) > 5): #Caso a quantidade de posts no grupo seja inferior a 5, a quantidade de SAMPLE será igual a quantidade de posts
    result, classific = classification(gp1.sample(5), lines) #Classifica o Grupo 0
else:
    result, classific = classification(gp1.sample(len(gp1)), lines) #Classifica o Grupo 0
print(result)
if(result):
    print('Vá direto para o grupo 2. Não precisa fazer do grupo 0 nesta etapa')

True
Vá direto para o grupo 2. Não precisa fazer do grupo 0 nesta etapa


In [71]:
dictlist = clusterizar(gp1)
df_aux = insert_column(gp1, dictlist)
gp0 = get_group(0,df_aux)
gp1 = get_group(1,df_aux)
gp2 = pd.concat([gp2,get_group(2,df_aux)])
if(len(gp1) > 5): #Caso a quantidade de posts no grupo seja inferior a 5, a quantidade de SAMPLE será igual a quantidade de posts
    result, classific = classification(gp1.sample(5), lines) #Classifica o Grupo 0
else:
    result, classific = classification(gp1.sample(len(gp1)), lines) #Classifica o Grupo 0
print(result)

True


In [ ]:
if(len(gp1) >= 3):
    dictlist = clusterizar(gp1)
    df_aux = insert_column(gp1, dictlist)
    gp0 = pd.concat([gp0,get_group(0,df_aux)])
    gp1 = get_group(1,df_aux)
    gp2 = pd.concat([gp2,get_group(2,df_aux)])
    if(len(gp1) > 5): #Caso a quantidade de posts no grupo seja inferior a 5, a quantidade de SAMPLE será igual a quantidade de posts
        result, classific = classification(gp1.sample(5), lines) #Classifica o Grupo 0
    else:
        result, classific = classification(gp1.sample(len(gp1)), lines) #Classifica o Grupo 0
    print(result)
else:
    if(len(gp1) > 5): #Caso a quantidade de posts no grupo seja inferior a 5, a quantidade de SAMPLE será igual a quantidade de posts
        result, classific = classification(gp1.sample(5), lines) #Classifica o Grupo 0
    else:
        result, classific = classification(gp1.sample(len(gp1)), lines) #Classifica o Grupo 0
    print(result)

### Tudo OK GP 1

In [79]:
dfTF = insert_column_class(dfTF, gp1, classific)
gp1 = pd.DataFrame()
dfTF

,telejornal,traz,uma,entrevista,ao,vivo,com,o,candidato,governo,...,okapanegão,grpaesp,baracho,pmesp,cmt,brpae,sjrp,httpdlvritjtsdk,group,classification
1,0.009,0.009,0.009,0.009,0.017,0.009,0.009,0.009,0.009,0.009,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
2,0.000,0.000,0.005,0.000,0.000,0.000,0.005,0.005,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
3,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
4,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.007,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
5,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
842,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.021,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
843,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
844,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
845,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Neutro


## Classificar grupo 2

Primeira vez na classificação do Grupo 2, não clusteriza

In [80]:
if(len(gp2) > 5): #Caso a quantidade de posts no grupo seja inferior a 5, a quantidade de SAMPLE será igual a quantidade de posts
    result, classific = classification(gp2.sample(5), lines) #Classifica o Grupo 1
else:
    result, classific = classification(gp2.sample(len(gp2)), lines) #Classifica o Grupo 1
print(result)

True


Na Segunda vez, o grupo 0 e 1 vão ser sobrescrevidos, pois agora será formado um novo grupo 0 e 1

In [60]:
dictlist = clusterizar(gp2)
df_aux = insert_column(gp2, dictlist)
gp0 = get_group(0,df_aux)
gp1 = get_group(1,df_aux)
gp2 = get_group(2,df_aux)
if(len(gp2) > 5): #Caso a quantidade de posts no grupo seja inferior a 5, a quantidade de SAMPLE será igual a quantidade de posts
    result, classific = classification(gp2.sample(5), lines) #Classifica o Grupo 2
else:
    result, classific = classification(gp2.sample(len(gp2)), lines) #Classifica o Grupo 2
print(result)

False


A partir da terceira, os grupos 0 e 1 deverão ser concatenados

In [62]:
dictlist = clusterizar(gp2)
df_aux = insert_column(gp2, dictlist)
gp0 = pd.concat([gp0,get_group(0,df_aux)])
gp1 = pd.concat([gp1,get_group(1,df_aux)])
gp2 = get_group(2,df_aux)
if(len(gp2) > 5): #Caso a quantidade de posts no grupo seja inferior a 5, a quantidade de SAMPLE será igual a quantidade de posts
    result, classific = classification(gp2.sample(5), lines) #Classifica o Grupo 2
else:
    result, classific = classification(gp2.sample(len(gp2)), lines) #Classifica o Grupo 2
print(result)

True


### Tudo OK

In [82]:
dfTF = insert_column_class(dfTF, gp2, classific)
gp2 = pd.DataFrame()
dfTF

,telejornal,traz,uma,entrevista,ao,vivo,com,o,candidato,governo,...,okapanegão,grpaesp,baracho,pmesp,cmt,brpae,sjrp,httpdlvritjtsdk,group,classification
1,0.009,0.009,0.009,0.009,0.017,0.009,0.009,0.009,0.009,0.009,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
2,0.000,0.000,0.005,0.000,0.000,0.000,0.005,0.005,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
3,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
4,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.007,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
5,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
842,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.021,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
843,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
844,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
845,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Neutro


### Executar para verificar se as classificações foram salvas no DF

In [83]:
dfTF.loc[dfTF['classification'] != '']

,telejornal,traz,uma,entrevista,ao,vivo,com,o,candidato,governo,...,okapanegão,grpaesp,baracho,pmesp,cmt,brpae,sjrp,httpdlvritjtsdk,group,classification
1,0.009,0.009,0.009,0.009,0.017,0.009,0.009,0.009,0.009,0.009,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
2,0.000,0.000,0.005,0.000,0.000,0.000,0.005,0.005,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
3,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
4,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.007,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
5,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
842,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.021,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
843,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
844,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
845,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Neutro


In [84]:
dfTF.loc[dfTF['classification'] == 'Neutro']

,telejornal,traz,uma,entrevista,ao,vivo,com,o,candidato,governo,...,okapanegão,grpaesp,baracho,pmesp,cmt,brpae,sjrp,httpdlvritjtsdk,group,classification
7,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.000,0.005,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0,Neutro
9,0.0,0.0,0.000,0.0,0.009,0.0,0.0,0.004,0.004,0.009,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0,Neutro
11,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.004,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0,Neutro
14,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0,Neutro
16,0.0,0.0,0.009,0.0,0.000,0.0,0.0,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Neutro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0,Neutro
838,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Neutro
839,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.000,0.000,...,0.008,0.008,0.008,0.008,0.008,0.008,0.008,0.008,2,Neutro
841,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0,Neutro


In [85]:
dfTF.loc[dfTF['classification'] == 'Negativo']

,telejornal,traz,uma,entrevista,ao,vivo,com,o,candidato,governo,...,okapanegão,grpaesp,baracho,pmesp,cmt,brpae,sjrp,httpdlvritjtsdk,group,classification
45,0.0,0.0,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,Negativo
47,0.0,0.0,0.000,0.000,0.000,0.0,0.008,0.004,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,Negativo
49,0.0,0.0,0.000,0.000,0.000,0.0,0.030,0.030,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,Negativo
71,0.0,0.0,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,Negativo
106,0.0,0.0,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,Negativo
136,0.0,0.0,0.000,0.000,0.000,0.0,0.000,0.004,0.000,0.008,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,Negativo
150,0.0,0.0,0.000,0.000,0.000,0.0,0.004,0.011,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,Negativo
165,0.0,0.0,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,Negativo
198,0.0,0.0,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,Negativo
226,0.0,0.0,0.000,0.000,0.000,0.0,0.000,0.005,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,Negativo


In [86]:
dfTF.loc[dfTF['classification'] == 'Positivo']

,telejornal,traz,uma,entrevista,ao,vivo,com,o,candidato,governo,...,okapanegão,grpaesp,baracho,pmesp,cmt,brpae,sjrp,httpdlvritjtsdk,group,classification
1,0.009,0.009,0.009,0.009,0.017,0.009,0.009,0.009,0.009,0.009,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
2,0.000,0.000,0.005,0.000,0.000,0.000,0.005,0.005,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
3,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
4,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.007,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
5,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
840,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.011,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
842,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.021,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
843,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
844,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo


In [87]:
dfTF

,telejornal,traz,uma,entrevista,ao,vivo,com,o,candidato,governo,...,okapanegão,grpaesp,baracho,pmesp,cmt,brpae,sjrp,httpdlvritjtsdk,group,classification
1,0.009,0.009,0.009,0.009,0.017,0.009,0.009,0.009,0.009,0.009,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
2,0.000,0.000,0.005,0.000,0.000,0.000,0.005,0.005,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
3,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
4,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.007,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
5,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
842,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.021,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
843,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
844,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Positivo
845,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2,Neutro


## Salvar DF da Vetorização com as classificações

In [90]:
writeCSV(dfTF, "Classificacao-Vetorizacao")

## Incluir coluna da classificação no DF dos posts

In [ ]:
df_posts = pd.read_csv("..\\DataBase\\Governamentais.txt")

In [91]:
filepath = "..\\DataBase\\Governamentais.txt"
word_count = {}
lines = []


with open(filepath, 'r', encoding="utf8") as fi:
    for line in fi:
        line = remove_punctuations(line)
        lines.append(line)

        
df_posts = pd.DataFrame(lines)

In [95]:
df_posts['classification'] = dfTF['classification'].values

In [96]:
df_posts

,0,classification
0,Telejornal traz uma entrevista ao vivo com o c...,Positivo
1,No sudeste estou fechado com o PLIrei apoiar u...,Positivo
2,Magno Malta Senador ESCarlos Manato Governador...,Positivo
3,Quem souber quais candidatos do seu estado que...,Positivo
4,Quem são nossos deputados estaduais Até agora ...,Positivo
...,...,...
841,Eu disse que ele sabe o que faz mas o Vinícius...,Positivo
842,Ta Vinicius Veny Sousa seu capitão vai voltar ...,Positivo
843,Alberto BeviniévisqueVinicius SouzaCapitão Wag...,Positivo
844,CTTU faz fiscalização na Capitão Rebelinho e r...,Neutro


In [97]:
writeCSV(dfTF, "Classificacao-Posts")